In [1]:
get_ipython().system_raw("mlflow ui --port 5090 &")

In [2]:
from pyngrok import ngrok

In [3]:
ngrok.kill()

[2025-02-03 03:30:03 +0000] [530] [INFO] Starting gunicorn 23.0.0
[2025-02-03 03:30:03 +0000] [530] [INFO] Listening at: http://127.0.0.1:5090 (530)
[2025-02-03 03:30:03 +0000] [530] [INFO] Using worker: sync
[2025-02-03 03:30:03 +0000] [531] [INFO] Booting worker with pid: 531
[2025-02-03 03:30:03 +0000] [532] [INFO] Booting worker with pid: 532
[2025-02-03 03:30:03 +0000] [533] [INFO] Booting worker with pid: 533
[2025-02-03 03:30:03 +0000] [534] [INFO] Booting worker with pid: 534


In [4]:
!ngrok config add-authtoken 2sViGViQ9YXhZ93JPXnt9zp2kSn_5tS656cBtfCP4acM4JTkb

Authtoken saved to configuration file: /home/workbench/.config/ngrok/ngrok.yml                      


In [5]:
ngrok_tunnel = ngrok.connect(addr="5090", proto="http", bind_tls=True)

t=2025-02-03T03:30:12+0000 lvl=warn msg="failed to check for update" obj=updater err="Post \"https://update.equinox.io/check\": context deadline exceeded"


In [7]:
print("MLflow UI ", ngrok_tunnel.public_url)

MLflow UI  https://5c6e-67-254-220-141.ngrok-free.app


In [8]:
import torch
import torch.nn as nn
from torch.nn import functional as F

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

import mlflow
import mlflow.pytorch

In [9]:
mlflow.set_experiment(experiment_name="mlflow pytorch demo")
    
dataset = pd.read_csv('https://raw.githubusercontent.com/futurexskill/ml-model-deployment/main/storepurchasedata_large.csv')
dataset.describe()
dataset.head()

X = dataset.iloc[:, :-1].values
y = dataset.iloc[:,-1].values


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size =.20,random_state=0)

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

Xtrain_ = torch.from_numpy(X_train).float()
Xtest_ = torch.from_numpy(X_test).float()

Xtrain_

ytrain_ = torch.from_numpy(y_train)
ytest_ = torch.from_numpy(y_test)

ytrain_

Xtrain_.shape, ytrain_.shape

Xtest_.shape, ytest_.shape

input_size=2
output_size=2
hidden_size=10

class Net(nn.Module):
   def __init__(self):
       super(Net, self).__init__()
       self.fc1 = torch.nn.Linear(input_size, hidden_size)
       self.fc2 = torch.nn.Linear(hidden_size, hidden_size)
       self.fc3 = torch.nn.Linear(hidden_size, output_size)


   def forward(self, X):
       X = torch.relu((self.fc1(X)))
       X = torch.relu((self.fc2(X)))
       X = self.fc3(X)

       return F.log_softmax(X,dim=1)

model = Net()

optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
loss_fn = nn.NLLLoss()

epochs = 150

for epoch in range(epochs):
  optimizer.zero_grad()
  Ypred = model(Xtrain_)
  loss = loss_fn(Ypred,  ytrain_)
  loss.backward()
  optimizer.step()
  print('Epoch',epoch, 'loss',loss.item())

mlflow.end_run()
with mlflow.start_run() as run:
    mlflow.log_param("epochs", 150)
    mlflow.pytorch.log_model(model, "models")
    mlflow.log_metric("loss", loss.item())

Epoch 0 loss 0.7213798761367798
Epoch 1 loss 0.6947463750839233
Epoch 2 loss 0.6685854196548462
Epoch 3 loss 0.643220841884613
Epoch 4 loss 0.618878185749054
Epoch 5 loss 0.5953922867774963
Epoch 6 loss 0.5716016888618469
Epoch 7 loss 0.5472028255462646
Epoch 8 loss 0.522203266620636
Epoch 9 loss 0.496574729681015
Epoch 10 loss 0.4701923429965973
Epoch 11 loss 0.44324055314064026
Epoch 12 loss 0.4165537357330322
Epoch 13 loss 0.39010781049728394
Epoch 14 loss 0.3645598888397217
Epoch 15 loss 0.3399718701839447
Epoch 16 loss 0.3166005611419678
Epoch 17 loss 0.29524558782577515
Epoch 18 loss 0.275443434715271
Epoch 19 loss 0.2569441795349121
Epoch 20 loss 0.2398807555437088
Epoch 21 loss 0.22437231242656708
Epoch 22 loss 0.21044106781482697
Epoch 23 loss 0.19811205565929413
Epoch 24 loss 0.18703149259090424
Epoch 25 loss 0.1770717352628708
Epoch 26 loss 0.16818960011005402
Epoch 27 loss 0.16036830842494965
Epoch 28 loss 0.15349934995174408
Epoch 29 loss 0.14760369062423706
Epoch 30 loss 

2025/02/03 03:30:38 WARNING mlflow.utils.requirements_utils: Found torch version (2.1.0a0+32f93b1) contains a local version label (+32f93b1). MLflow logged a pip requirement for this package as 'torch==2.1.0a0' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.
2025/02/03 03:30:45 WARNING mlflow.utils.requirements_utils: Found torch version (2.1.0a0+32f93b1) contains a local version label (+32f93b1). MLflow logged a pip requirement for this package as 'torch==2.1.0a0' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.
2025/02/03 03:30:45 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


In [12]:
ngrok.kill()